In [3]:
import json
import pandas as pd
pd.set_option('display.max_rows', 40)

In [5]:
import random

In [13]:
import sys, json
from pathlib import Path

# 1. Projekt-Root nur einmal bestimmen
PROJECT_ROOT = Path.cwd().parent.resolve()

# 2. sys.path fürs Importieren um PROJECT_ROOT erweitern
sys.path.insert(0, str(PROJECT_ROOT))

In [27]:
import utils.presenter as show
import utils.rolling_planning as rp

In [28]:
data_file = Path(PROJECT_ROOT, "data/jobshop_instances.json")

# Datei laden
with open(data_file, "r", encoding="utf-8") as f:
    jobshop_instances = json.load(f)

instance =  jobshop_instances["instance ft10"]
show.print_jobs(instance)

job 0:  [[0, 29], [1, 78], [2, 9], [3, 36], [4, 49], [5, 11], [6, 62], [7, 56], [8, 44], [9, 21]]
job 1:  [[0, 43], [2, 90], [4, 75], [9, 11], [3, 69], [1, 28], [6, 46], [5, 46], [7, 72], [8, 30]]
job 2:  [[1, 91], [0, 85], [3, 39], [2, 74], [8, 90], [5, 10], [7, 12], [6, 89], [9, 45], [4, 33]]
job 3:  [[1, 81], [2, 95], [0, 71], [4, 99], [6, 9], [8, 52], [7, 85], [3, 98], [9, 22], [5, 43]]
job 4:  [[2, 14], [0, 6], [1, 22], [5, 61], [3, 26], [4, 69], [8, 21], [7, 49], [9, 72], [6, 53]]
job 5:  [[2, 84], [1, 2], [5, 52], [3, 95], [8, 48], [9, 72], [0, 47], [6, 65], [4, 6], [7, 25]]
job 6:  [[1, 46], [0, 37], [3, 61], [2, 13], [6, 32], [5, 21], [9, 32], [8, 89], [7, 30], [4, 55]]
job 7:  [[2, 31], [0, 86], [1, 46], [5, 74], [4, 32], [6, 88], [8, 19], [9, 48], [7, 36], [3, 79]]
job 8:  [[0, 76], [1, 69], [3, 76], [5, 51], [2, 85], [9, 11], [6, 40], [7, 89], [4, 26], [8, 74]]
job 9:  [[1, 85], [0, 13], [2, 61], [6, 7], [8, 64], [9, 76], [5, 47], [3, 52], [4, 90], [7, 45]]



In [30]:
jssp_data, df_times = rp.init_jobs_with_arrivals(instance, 5, u_b_mmax = 0.75, 
                                                    generate_deadlines = True, deadlines_buffer_factor= 1.8)
show.print_jobs(jssp_data)

Job_000:  [[0, 29], [1, 78], [2, 9], [3, 36], [4, 49], [5, 11], [6, 62], [7, 56], [8, 44], [9, 21]]
Job_001:  [[0, 43], [2, 90], [4, 75], [9, 11], [3, 69], [1, 28], [6, 46], [5, 46], [7, 72], [8, 30]]
Job_002:  [[1, 91], [0, 85], [3, 39], [2, 74], [8, 90], [5, 10], [7, 12], [6, 89], [9, 45], [4, 33]]
Job_003:  [[1, 81], [2, 95], [0, 71], [4, 99], [6, 9], [8, 52], [7, 85], [3, 98], [9, 22], [5, 43]]
Job_004:  [[2, 14], [0, 6], [1, 22], [5, 61], [3, 26], [4, 69], [8, 21], [7, 49], [9, 72], [6, 53]]
Job_005:  [[2, 84], [1, 2], [5, 52], [3, 95], [8, 48], [9, 72], [0, 47], [6, 65], [4, 6], [7, 25]]
Job_006:  [[1, 46], [0, 37], [3, 61], [2, 13], [6, 32], [5, 21], [9, 32], [8, 89], [7, 30], [4, 55]]
Job_007:  [[2, 31], [0, 86], [1, 46], [5, 74], [4, 32], [6, 88], [8, 19], [9, 48], [7, 36], [3, 79]]
Job_008:  [[0, 76], [1, 69], [3, 76], [5, 51], [2, 85], [9, 11], [6, 40], [7, 89], [4, 26], [8, 74]]
Job_009:  [[1, 85], [0, 13], [2, 61], [6, 7], [8, 64], [9, 76], [5, 47], [3, 52], [4, 90], [7, 4

In [32]:
df_times

,Job,Arrival,Deadline
0,Job_000,14.37,2238.967789
1,Job_001,116.28,2988.545499
2,Job_002,141.78,3340.695301
3,Job_003,192.34,3881.230004
4,Job_004,236.45,2449.784002
...,...,...,...
110,Job_110,6629.15,8842.484002
111,Job_111,6787.00,9822.590400
112,Job_112,6941.64,9166.237789
113,Job_113,6990.12,9862.385499


## Fenster-Shuffle (Block- oder „window“-Shuffle)

In [36]:
from random import shuffle

def window_shuffle_df_rows(df: pd.DataFrame, k: int = 3, reset_index: bool = False) -> pd.DataFrame:
    """
    Teilt den DataFrame in Blöcke à k Zeilen und vertauscht innerhalb
    jedes Blocks die Zeilen in zufälliger Reihenfolge.
    """
    df = df.copy()                       # Original nicht verändern
    blocks = []

    for start in range(0, len(df), k):
        block_idx = list(range(start, min(start + k, len(df))))
        shuffle(block_idx)               # Reihenfolge im Block mischen
        blocks.append(df.iloc[block_idx])

    out = pd.concat(blocks)
    if reset_index:
        out.reset_index(drop=True, inplace=True)
    return out

In [42]:
df_times_shuffled = window_shuffle_df_rows(df_times, k=8, reset_index=True)
df_times_shuffled

,Job,Arrival,Deadline
0,Job_007,409.92,3445.510400
1,Job_005,316.00,3109.418995
2,Job_004,236.45,2449.784002
3,Job_001,116.28,2988.545499
4,Job_006,389.61,2732.477544
...,...,...,...
110,Job_106,6447.70,10136.590004
111,Job_104,6407.75,8632.347789
112,Job_114,7090.59,10289.505301
113,Job_113,6990.12,9862.385499


## Index stören + sortieren (Perturb-and-sort)

In [47]:
def jitter_shuffle_df_rows(df: pd.DataFrame, sigma: float = 1.0, reset_index: bool = False,
                           rng: random.Random | None = None) -> pd.DataFrame:
    """
    Sortiert die Zeilen nach leicht verrauschten Positions-Keys.
    
    Parameters
    ----------
    df : DataFrame
        Eingabe-Frame.
    sigma : float
        Standardabweichung des Gauß-Rauschens; je größer, desto stärker die Durchmischung.
    reset_index : bool
        Optional die Indizes neu von 0‥n-1 setzen.
    rng : random.Random | None
        Eigene Random-Instanz für reproduzierbare Ergebnisse (seedbar).
    """
    if rng is None:
        rng = random
    noisy_keys = [i + rng.gauss(0, sigma) for i in range(len(df))]
    out = df.iloc[pd.Series(noisy_keys).sort_values().index]
    if reset_index:
        out = out.reset_index(drop=True)
    return out

In [49]:
rng = random.Random(42)                   # fester Seed
df_times_jitter_shuffled = jitter_shuffle_df_rows(df_times, sigma=2.8, rng=rng)
df_times_jitter_shuffled.head(20).reset_index(drop=True)

,Job,Arrival,Deadline
0,Job_000,14.37,2238.967789
1,Job_001,116.28,2988.545499
2,Job_005,316.00,3109.418995
3,Job_002,141.78,3340.695301
4,Job_004,236.45,2449.784002
5,Job_003,192.34,3881.230004
6,Job_007,409.92,3445.510400
7,Job_006,389.61,2732.477544
8,Job_008,427.52,3789.760202
9,Job_009,529.40,3570.622293


## 3. Mallows-Permutation (kontrollierte Inversions-Wahrscheinlichkeit)

In [52]:
def _mallows_geom_cdf(i: int, q: float) -> list[float]:
    """
    Vorberechnete kumulative Wahrscheinlichkeiten P(D ≤ d) für d = 0..i-1.
    Spart Zeit, wenn man viele Permutationen mit identischen (i,q) zieht.
    """
    norm = (1 - q**i) / (1 - q) if q != 1 else i      # ∑_{d=0}^{i-1} q^d
    cdf, cum = [], 0.0
    for d in range(i):
        cum += (q**d if q != 1 else 1) / norm
        cdf.append(cum)
    return cdf


def mallows_identity_perm(n: int, q: float = 0.5,
                          rng: random.Random | None = None) -> list[int]:
    """
    Liefert eine Mallows-Permutationsliste der Länge n, zentriert um die Identität.
    """
    if rng is None:
        rng = random
    perm = []
    for i in range(1, n + 1):
        r = rng.random()
        cdf = _mallows_geom_cdf(i, q)
        # kleinste d mit CDF ≥ r
        for d, p in enumerate(cdf):
            if r <= p:
                break
        perm.insert(i - 1 - d, i)
    return perm


def mallows_shuffle_df_rows(df: pd.DataFrame, q: float = 0.5,
                            reset_index: bool = False,
                            rng: random.Random | None = None) -> pd.DataFrame:
    """
    Sortiert DataFrame-Zeilen nach einer Mallows-Permutation (Identitätszentrum).

    ▸ q < 1  → permutiert 'nah' an der Originalreihenfolge  
    ▸ q = 1  → echte Zufallspermutation (uniform)  
    ▸ q > 1  → zieht stärker Richtung *umgekehrter* Reihenfolge
    """
    if rng is None:
        rng = random
    perm = mallows_identity_perm(len(df), q=q, rng=rng)
    out = df.iloc[[p - 1 for p in perm]]      # perm ist 1-basiert
    if reset_index:
        out = out.reset_index(drop=True)
    return out

In [54]:
print("q = 0.6  → leichte Durchmischung")
df_times_mallows_shuffled = mallows_shuffle_df_rows(df_times, q=0.6, reset_index=True)
df_times_mallows_shuffled

q = 0.6  → leichte Durchmischung


,Job,Arrival,Deadline
0,Job_007,409.92,3445.510400
1,Job_001,116.28,2988.545499
2,Job_000,14.37,2238.967789
3,Job_003,192.34,3881.230004
4,Job_002,141.78,3340.695301
...,...,...,...
110,Job_110,6629.15,8842.484002
111,Job_113,6990.12,9862.385499
112,Job_111,6787.00,9822.590400
113,Job_112,6941.64,9166.237789


## 4. Zufällige benachbarte Vertauschungen (adjacent-swap walk)

In [57]:
def adjacent_swaps_perm(n: int, T: int = 5,
                        rng: random.Random | None = None) -> list[int]:
    """
    Liefert eine 0-basierte Permutation der Länge *n*, erzeugt durch *T*
    zufällige Adjacent Swaps.

    ▸ T klein  → Permutation dicht an der Identität  
    ▸ T ≫ n    → nähert sich (langsam!) einer Uniform­permutation
    """
    if rng is None:
        rng = random
    perm = list(range(n))
    for _ in range(T):
        i = rng.randrange(n - 1)
        perm[i], perm[i + 1] = perm[i + 1], perm[i]
    return perm


def adjacent_swaps_df_rows(df: pd.DataFrame, T: int = 5,
                           reset_index: bool = False,
                           rng: random.Random | None = None) -> pd.DataFrame:
    """
    Vertauscht Zeilen in *T* zufälligen benachbarten Paaren.

    Example
    -------
    >>> shuffled = adjacent_swaps_df_rows(df, T=6, reset_index=True)
    """
    perm = adjacent_swaps_perm(len(df), T=T, rng=rng)
    out = df.iloc[perm]
    if reset_index:
        out = out.reset_index(drop=True)
    return out


# ── (optional) innerhalb jeder Zeile spaltenweise ────────────────────
def adjacent_swaps_series(s: pd.Series, T: int = 5,
                          rng: random.Random | None = None) -> pd.Series:
    """
    Wendet *T* zufällige Adjacent Swaps auf eine Series (eine Zeile) an.
    """
    if rng is None:
        rng = random
    idx = adjacent_swaps_perm(len(s), T=T, rng=rng)
    return s.iloc[idx]


def adjacent_swaps_df_cols(df: pd.DataFrame, T: int = 5,
                           rng: random.Random | None = None) -> pd.DataFrame:
    """
    Tauscht in *jeder* Zeile jeweils *T* benachbarte Spaltenwerte.
    """
    return df.apply(adjacent_swaps_series, axis=1, T=T, rng=rng)

In [59]:
rng = random.Random(42)          # reproduzierbar
shuffled = adjacent_swaps_df_rows(df_times, T=6, reset_index=True, rng=rng)
shuffled.head(25)

,Job,Arrival,Deadline
0,Job_000,14.37,2238.967789
1,Job_001,116.28,2988.545499
2,Job_002,141.78,3340.695301
3,Job_004,236.45,2449.784002
4,Job_003,192.34,3881.230004
5,Job_005,316.00,3109.418995
6,Job_006,389.61,2732.477544
7,Job_007,409.92,3445.510400
8,Job_008,427.52,3789.760202
9,Job_009,529.40,3570.622293
